In [87]:
# Python script to read stock info from csv file

import pandas as pd;
import os;
import psycopg2;

In [142]:
# Connect to RDS

# Retrieve RDS log in info from text file
file = open('../private.txt', 'r');
lines = file.readlines();

for i,l in enumerate(lines):
    if l[-1] == '\n':
        lines[i] = l[0:-1];
        
database_t = lines[2];
user_t = lines[3];
password_t = lines[4];
host_t = lines[5];
port_t = lines[6];

# Try Connection
try:
    db = psycopg2.connect(
        database= database_t,
        user= user_t,
        password= password_t,
        host= host_t,
        port= port_t
    )
except:
    print("ERROR: Unable to connect to the RDS server.")
    

cur = db.cursor()
cur.execute("""SELECT table_name FROM information_schema.tables
       WHERE table_schema = 'public'""")

for table in cur.fetchall():
    print(table)


('users',)
('investments',)
('stocks',)
('transactions',)
('prices',)


In [143]:
meta_path = '../data/stock_market_dataset/symbols_valid_meta.csv'
meta_data = pd.read_csv(meta_path)

print(meta_data[meta_data["Symbol"] == "AA"]["Security Name"].shape == (1,))

print(meta_data[meta_data["Symbol"] == "AA"]["Security Name"].iloc[0])



True
Alcoa Corporation Common Stock 


In [144]:
# Read CSV files

path = '../data/stock_market_dataset/stocks'

files = os.listdir(path)

# Skip first two files which have already been inserted into DB
# files = files[2:]

# TICKER.csv
#     0 Date
#     1 Open
#     2 High
#     3 Low
#     4 Close
#     5 Adj Close
#     6 Volume

error_output_file = open("error.txt", "a")

for f in files:
    fpath = '../data/stock_market_dataset/stocks/'+ f
    data = pd.read_csv(fpath)
    
    ticker = f[:-4]
    
    print(ticker)
    if(meta_data[meta_data["Symbol"] == ticker]["Security Name"].shape != (1,)):
        error_output_file.write(f)
        break
    
    name = meta_data[meta_data["Symbol"] == ticker]["Security Name"].iloc[0]
    
    squery = """INSERT INTO stocks(ticker, stock_name) VALUES('%s', '%s');"""%(ticker,name);
    cur.execute(squery);
    db.commit();
    
    args_str = ','.join(cur.mogrify("(%s,%s,%s,%s,%s,%s,%s,%s)", (ticker,) + row).decode("utf-8") for row in data.itertuples(index=False))
    cur.execute("INSERT INTO prices (ticker, date, open, high, low, close, adj_close, volume) VALUES " + args_str)
    
#     for row in data.itertuples(index=False):
#         date = row[0]
#         opn = str(row[1])
#         high = str(row[2])
#         low = str(row[3])
#         close = str(row[4])
#         adj = str(row[5])
#         vol = str(row[6])
        
        
#         query = """INSERT INTO prices (ticker, date, open, high, low, close, adj_close, volume) 
#         VALUES ('%s', '%s', %s, %s, %s, %s, %s, %s);"""%(ticker, date, opn, high, low, close, adj, vol);       
#         cur.execute(pquery);
    db.commit();
    
    print(f, ticker, name)



A
A.csv A Agilent Technologies, Inc. Common Stock
AA
AA.csv AA Alcoa Corporation Common Stock 
AACG
AACG.csv AACG ATA Creativity Global - American Depositary Shares, each representing two common shares
AAL
AAL.csv AAL American Airlines Group, Inc. - Common Stock
AAMC
AAMC.csv AAMC Altisource Asset Management Corp Com
AAME
AAME.csv AAME Atlantic American Corporation - Common Stock
AAN


SyntaxError: syntax error at or near "s"
LINE 1: ...O stocks(ticker, stock_name) VALUES('AAN', 'Aaron's, Inc. Co...
                                                             ^
